# 📁 File Operations Using Code Interpreter

Welcome to the File Operations module using AWS Bedrock AgentCore Code Interpreter! This notebook focuses on JSON and CSV file generation using Deno APIs with simple, practical examples.

## 🎯 Learning Objectives

In this notebook, you'll learn how to:

- 🔧 Set up and use AgentCore Code Interpreter for file operations
- 📄 Create and manipulate JSON files with structured data
- 📊 Generate CSV files for tabular data
- 🔄 Use both JavaScript (Deno) and Python for file I/O operations
- 🏗️ Implement file operations using both boto3 and AgentCore SDK

## 📋 Prerequisites

- AWS account with access to Amazon Bedrock AgentCore
- Proper IAM permissions for AgentCore services
- Python 3.12 environment with required dependencies

Let's begin! 🚀

## 🔧 Section 1: Environment Setup

In [ ]:
# 🔧 Environment Setup Instructions

# IMPORTANT: This notebook requires Python 3.12.8 as specified in pyproject.toml
# All dependencies are managed via UV (recommended approach for this project)

# SETUP STEPS:
# 1. Create virtual environment: uv venv
# 2. Install all dependencies: uv sync
# 3. Launch Jupyter Lab: uv run jupyter lab

# FOR VS CODE USERS:
# 1. Run: uv run python -c "import sys; print(sys.executable)"
# 2. Copy the path and select it as your Python interpreter in VS Code
# 3. The kernel should now have access to all required dependencies

# Verify environment setup
import sys
print(f"✅ Python version: {sys.version}")
print(f"✅ Python executable: {sys.executable}")
print("✅ Environment setup complete! All dependencies should be available via UV.")

In [ ]:
# Import core dependencies
import boto3
import json
import os
import time
from typing import Dict, Any, List, Optional

# Import AgentCore SDK
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter, code_session

# Import existing Module 3 components
from agentic_platform.core.models.memory_models import Message, SessionContext
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse

print("✅ Dependencies imported successfully!")

In [ ]:
# Initialize AWS clients with specific profile
profile_name = "ags-tech-namer-ca-pace+ai3-core-Admin"
session = boto3.Session(profile_name=profile_name)
region_name = "us-west-2"

# Verify credentials are working
try:
    credentials = session.get_credentials()
    print(f"✅ Using AWS profile: {profile_name}")
    print(f"✅ Access Key: {credentials.access_key[:10]}...")
    print(f"✅ Region: {region_name}")
except Exception as e:
    print(f"❌ Error with AWS credentials: {e}")
    raise

# Standard AgentCore client
bedrock_agentcore_client = session.client(
    'bedrock-agentcore',
    region_name=region_name,
    endpoint_url=f"https://bedrock-agentcore.{region_name}.amazonaws.com"
)

print("✅ AWS clients initialized successfully with profile!")

## 📄 Section 2: JSON File Operations with Boto3

Let's start with creating and reading JSON files using JavaScript (Deno) APIs.

In [ ]:
# JavaScript code for JSON file operations using Deno
javascript_json_code = """
// JSON File Operations Example
console.log('Starting JSON file operations...');

// Simple user profile data
const userProfile = {
    id: 'USR-001',
    name: 'John Doe',
    email: 'john.doe@example.com',
    age: 28,
    preferences: {
        theme: 'dark',
        notifications: true,
        language: 'en'
    },
    createdAt: new Date().toISOString()
};

// Convert to JSON string with formatting
const jsonString = JSON.stringify(userProfile, null, 2);

console.log('\n📝 User Profile JSON:');
console.log(jsonString);

// Write JSON to file (Deno file system API)
try {
    // In Deno, we would use:
    // await Deno.writeTextFile('user_profile.json', jsonString);
    
    // For demonstration, we'll simulate the operation
    console.log('\n✅ JSON file would be written to: user_profile.json');
    console.log('File size: ' + jsonString.length + ' bytes');
    
} catch (error) {
    console.error('Error writing file:', error);
}

// Parse JSON back to object
const parsedProfile = JSON.parse(jsonString);

// Validate the data
console.log('\n🔍 Validation Results:');
console.log('User ID: ' + parsedProfile.id);
console.log('User Name: ' + parsedProfile.name);
console.log('Email Valid: ' + parsedProfile.email.includes('@'));
console.log('Theme Setting: ' + parsedProfile.preferences.theme);

// Create a simple configuration file
const appConfig = {
    version: '1.0.0',
    apiEndpoint: 'https://api.example.com',
    maxRetries: 3,
    timeout: 5000,
    features: ['auth', 'dashboard', 'reports']
};

const configJson = JSON.stringify(appConfig, null, 2);

console.log('\n⚙️ Application Configuration:');
console.log(configJson);

console.log('\n✅ JSON operations completed successfully!');

// Return summary
{
    filesCreated: ['user_profile.json', 'app_config.json'],
    totalSize: jsonString.length + configJson.length,
    status: 'success'
};
"""

print("JavaScript JSON operations code prepared!")

In [ ]:
def execute_code_with_boto3(code: str, language: str = "javascript") -> Dict[str, Any]:
    """
    Execute code using boto3 direct API calls to AgentCore Code Interpreter.
    
    Args:
        code: The code to execute
        language: Programming language (javascript or python)
    
    Returns:
        Dictionary with execution results
    """
    try:
        # Start a code interpreter session
        session_response = bedrock_agentcore_client.start_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            name="boto3-file-ops-session",
            sessionTimeoutSeconds=300
        )
        session_id = session_response["sessionId"]
        print(f"✅ Started session: {session_id}")
        
        # Execute the code
        execute_response = bedrock_agentcore_client.invoke_code_interpreter(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id,
            name="executeCode",
            arguments={
                "language": language,
                "code": code
            }
        )
        
        # Process the response stream
        results = []
        for event in execute_response['stream']:
            if 'result' in event:
                result = event['result']
                results.append(result)
                
                # Print text output
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(f"Output: {content_item['text']}")
        
        # Clean up session
        bedrock_agentcore_client.stop_code_interpreter_session(
            codeInterpreterIdentifier="aws.codeinterpreter.v1",
            sessionId=session_id
        )
        print(f"✅ Session {session_id} stopped")
        
        return {
            "success": True,
            "session_id": session_id,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

In [ ]:
# Execute the JavaScript JSON operations code using boto3
print("🚀 Executing JavaScript JSON file operations with boto3...\n")
boto3_json_result = execute_code_with_boto3(javascript_json_code, "javascript")
print(f"\n✅ Execution completed: {boto3_json_result['success']}")

## 📊 Section 3: CSV File Operations with Boto3

Now let's work with CSV files using JavaScript (Deno).

In [ ]:
# JavaScript code for CSV file operations
javascript_csv_code = """
// CSV File Operations Example
console.log('Starting CSV file operations...');

// Simple product data
const products = [
    { id: 'P001', name: 'Laptop', price: 999.99, stock: 15 },
    { id: 'P002', name: 'Mouse', price: 29.99, stock: 50 },
    { id: 'P003', name: 'Keyboard', price: 79.99, stock: 30 }
];

// Function to convert array of objects to CSV
function arrayToCSV(data) {
    if (data.length === 0) return '';
    
    // Get headers from first object
    const headers = Object.keys(data[0]);
    const csvHeaders = headers.join(',');
    
    // Convert each object to CSV row
    const csvRows = data.map(obj => {
        return headers.map(header => {
            const value = obj[header];
            // Handle values that might contain commas
            return typeof value === 'string' && value.includes(',') 
                ? `"${value}"` 
                : value;
        }).join(',');
    });
    
    // Combine headers and rows
    return [csvHeaders, ...csvRows].join('\n');
}

// Generate CSV content
const csvContent = arrayToCSV(products);

console.log('\n📊 Generated CSV Content:');
console.log('================================');
console.log(csvContent);

// Calculate some statistics
const totalValue = products.reduce((sum, p) => sum + (p.price * p.stock), 0);
const avgPrice = products.reduce((sum, p) => sum + p.price, 0) / products.length;
const totalStock = products.reduce((sum, p) => sum + p.stock, 0);

console.log('\n📈 Inventory Statistics:');
console.log('Total inventory value: $' + totalValue.toFixed(2));
console.log('Average product price: $' + avgPrice.toFixed(2));
console.log('Total items in stock: ' + totalStock);

// Create a simple sales report CSV
const salesData = [
    { date: '2024-01-01', product: 'Laptop', quantity: 2, revenue: 1999.98 },
    { date: '2024-01-02', product: 'Mouse', quantity: 5, revenue: 149.95 },
    { date: '2024-01-03', product: 'Keyboard', quantity: 3, revenue: 239.97 }
];

const salesCSV = arrayToCSV(salesData);

console.log('\n💰 Sales Report CSV:');
console.log('================================');
console.log(salesCSV);

// Parse CSV back to verify
function parseCSV(csvString) {
    const lines = csvString.split('\n');
    const headers = lines[0].split(',');
    const data = [];
    
    for (let i = 1; i < lines.length; i++) {
        const values = lines[i].split(',');
        const obj = {};
        headers.forEach((header, index) => {
            obj[header] = values[index];
        });
        data.push(obj);
    }
    
    return data;
}

const parsedProducts = parseCSV(csvContent);
console.log('\n✅ CSV parsing verification:');
console.log('Parsed ' + parsedProducts.length + ' products successfully');

console.log('\n✅ CSV operations completed successfully!');

// Return summary
{
    filesGenerated: ['products.csv', 'sales_report.csv'],
    totalRows: products.length + salesData.length,
    totalBytes: csvContent.length + salesCSV.length,
    status: 'success'
};
"""

print("JavaScript CSV operations code prepared!")

In [ ]:
# Execute the JavaScript CSV operations code using boto3
print("🚀 Executing JavaScript CSV file operations with boto3...\n")
boto3_csv_result = execute_code_with_boto3(javascript_csv_code, "javascript")
print(f"\n✅ Execution completed: {boto3_csv_result['success']}")

## 🐍 Section 4: File Operations with AgentCore SDK

Now let's perform file operations using Python through the AgentCore SDK.

In [ ]:
# Python code for JSON and CSV file operations
python_file_ops_code = """
import json
import csv
from io import StringIO
from datetime import datetime

print('Starting file operations with Python...')

# ===== JSON Operations =====
print('\n📄 JSON File Operations:')
print('=' * 40)

# Create a simple employee record
employee = {
    'emp_id': 'EMP-2024-001',
    'name': 'Jane Smith',
    'department': 'Engineering',
    'salary': 85000,
    'skills': ['Python', 'AWS', 'Docker'],
    'active': True,
    'joined_date': '2024-01-15'
}

# Convert to formatted JSON
json_str = json.dumps(employee, indent=2)
print('Employee Record (JSON):')
print(json_str)

# Parse and validate
parsed_employee = json.loads(json_str)
print(f"\n✅ Validated: Employee {parsed_employee['name']} in {parsed_employee['department']}")
print(f"Skills count: {len(parsed_employee['skills'])}")

# Create a settings file
settings = {
    'app_name': 'DataProcessor',
    'version': '2.1.0',
    'debug_mode': False,
    'max_connections': 100,
    'database': {
        'host': 'localhost',
        'port': 5432,
        'name': 'app_db'
    }
}

settings_json = json.dumps(settings, indent=2)
print('\nApplication Settings:')
print(settings_json)

# ===== CSV Operations =====
print('\n📊 CSV File Operations:')
print('=' * 40)

# Create inventory data
inventory = [
    ['Item', 'Category', 'Quantity', 'Price'],
    ['Notebook', 'Stationery', 100, 2.50],
    ['Pen', 'Stationery', 250, 1.00],
    ['Calculator', 'Electronics', 25, 15.99]
]

# Write to CSV format
csv_buffer = StringIO()
csv_writer = csv.writer(csv_buffer)
for row in inventory:
    csv_writer.writerow(row)

csv_content = csv_buffer.getvalue()
print('Inventory CSV:')
print(csv_content)

# Parse CSV and calculate totals
csv_buffer.seek(0)
csv_reader = csv.DictReader(csv_buffer)

total_value = 0
total_items = 0

print('\n📈 Inventory Analysis:')
for row in csv_reader:
    quantity = float(row['Quantity'])
    price = float(row['Price'])
    value = quantity * price
    total_value += value
    total_items += quantity
    print(f"{row['Item']}: {quantity} units @ ${price:.2f} = ${value:.2f}")

print(f'\nTotal items: {int(total_items)}')
print(f'Total inventory value: ${total_value:.2f}')

# Create a transaction log CSV
transactions = [
    {'id': 'TXN001', 'date': '2024-01-10', 'amount': 150.00, 'status': 'completed'},
    {'id': 'TXN002', 'date': '2024-01-11', 'amount': 75.50, 'status': 'completed'},
    {'id': 'TXN003', 'date': '2024-01-12', 'amount': 200.00, 'status': 'pending'}
]

# Write transactions to CSV
txn_buffer = StringIO()
fieldnames = ['id', 'date', 'amount', 'status']
dict_writer = csv.DictWriter(txn_buffer, fieldnames=fieldnames)
dict_writer.writeheader()
dict_writer.writerows(transactions)

print('\n💰 Transaction Log CSV:')
print(txn_buffer.getvalue())

# Summary statistics
completed_amount = sum(t['amount'] for t in transactions if t['status'] == 'completed')
pending_amount = sum(t['amount'] for t in transactions if t['status'] == 'pending')

print('\n📊 Transaction Summary:')
print(f'Completed transactions: ${completed_amount:.2f}')
print(f'Pending transactions: ${pending_amount:.2f}')
print(f'Total transactions: {len(transactions)}')

print('\n✅ All file operations completed successfully!')

# Return summary
{
    'json_files': ['employee.json', 'settings.json'],
    'csv_files': ['inventory.csv', 'transactions.csv'],
    'total_operations': 4,
    'status': 'success'
}
"""

print("Python file operations code prepared!")

In [ ]:
def execute_code_with_sdk(code: str, language: str = "python") -> Dict[str, Any]:
    """
    Execute code using the AgentCore SDK (higher-level interface).
    
    Args:
        code: The code to execute
        language: Programming language (javascript or python)
    
    Returns:
        Dictionary with execution results
    """
    try:
        # Configure and start the code interpreter session
        code_client = CodeInterpreter(region_name)
        code_client.start()
        print("✅ AgentCore SDK session started")
        
        # Execute the code
        response = code_client.invoke("executeCode", {
            "language": language,
            "code": code
        })
        
        # Process and print the response
        results = []
        for event in response["stream"]:
            if "result" in event:
                result = event["result"]
                results.append(result)
                
                # Extract and display text content
                if 'content' in result:
                    for content_item in result['content']:
                        if content_item['type'] == 'text':
                            print(content_item['text'])
        
        # Clean up and stop the session
        code_client.stop()
        print("\n✅ AgentCore SDK session stopped")
        
        return {
            "success": True,
            "results": results
        }
        
    except Exception as e:
        print(f"❌ Error executing code with SDK: {str(e)}")
        return {
            "success": False,
            "error": str(e)
        }

In [ ]:
# Execute the Python file operations code using AgentCore SDK
print("🚀 Executing Python file operations with AgentCore SDK...\n")
sdk_file_result = execute_code_with_sdk(python_file_ops_code, "python")
print(f"\n✅ Execution completed: {sdk_file_result['success']}")

## 🎯 Section 5: Summary and Key Takeaways

### 📁 What We've Learned

In this notebook, we've successfully demonstrated:

1. **JSON File Operations**: 
   - Creating structured JSON data (user profiles, configurations)
   - Parsing and validating JSON content
   - Working with nested JSON objects

2. **CSV File Operations**:
   - Generating CSV from arrays of objects
   - Parsing CSV back to structured data
   - Handling different data types in CSV format

3. **Two Implementation Approaches**:
   - **Boto3**: Direct API calls with JavaScript/Deno file operations
   - **AgentCore SDK**: Python implementation with built-in CSV and JSON modules

### 🔑 Key Differences Between Languages

| Aspect | JavaScript (Deno) | Python |
|--------|-------------------|--------|
| **JSON Handling** | Native JSON.stringify/parse | json module |
| **CSV Support** | Manual implementation | Built-in csv module |
| **File I/O** | Deno.writeTextFile (async) | Built-in file operations |
| **Data Processing** | Array methods (map, reduce) | List comprehensions, csv.DictReader |

### 💡 Best Practices for File Operations

1. **Data Validation**: Always validate data before and after file operations
2. **Error Handling**: Include try-catch blocks for file I/O operations
3. **Format Selection**: 
   - Use JSON for hierarchical/nested data
   - Use CSV for tabular/flat data
4. **Memory Efficiency**: Use streaming for large files
5. **Encoding**: Be explicit about character encoding (UTF-8)

### 🚀 Next Steps

- Explore the companion notebook on data processing and analysis
- Try working with larger datasets
- Implement file upload/download in agent workflows
- Experiment with other file formats (XML, YAML)
- Integrate file operations with S3 for persistent storage

### 📚 Additional Resources

- [AWS Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/)
- [Deno File System API](https://deno.land/manual/runtime/file_system)
- [Python CSV Module Documentation](https://docs.python.org/3/library/csv.html)
- [Module 3 Other Notebooks](../README.md)

Happy coding! 🎉